In [123]:
#manejo bbdd
import pandas as pd
import numpy as np

#gráficas
import seaborn as sns
import matplotlib.pyplot as plt

#estadística
import math
from scipy.stats import norm
from scipy import stats #para box-cox entre otros
from scipy.stats import skew


#preprocesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder


#modelos y evaluación
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error,r2_score

# OPCIÓN 1 ELIMINANDO LA MULTICOLINEALIDAD

In [124]:
df = pd.read_csv("StudentsMathScoresAnalisis.csv")

In [125]:
df

,NrSiblings,MathScore,ParentEduc,PracticeSport,WklyStudyHours,Gender_male,EthnicGroup_group B,EthnicGroup_group C,EthnicGroup_group D,EthnicGroup_group E,LunchType_standard,TestPrep_none,IsFirstChild_yes
0,3,71,3,2,0,False,False,True,False,False,True,True,True
1,0,69,1,1,1,False,False,True,False,False,True,True,True
2,4,87,4,1,0,False,True,False,False,False,True,True,True
3,1,45,2,0,1,True,False,False,False,False,False,True,False
4,0,76,1,1,1,True,False,True,False,False,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28556,2,59,0,1,1,False,False,False,True,False,True,True,False
28557,1,58,0,2,1,True,False,False,False,True,True,True,False
28558,1,61,0,1,1,False,False,True,False,False,False,False,False
28559,3,82,2,2,1,False,False,False,True,False,True,False,False


In [126]:
# tenemos que definir que es cada variable
# variable target la llamaremos y, es nuestra variable objetivo(la variable dependiente), la que quiero predecir /
# / la quito del dataframe de las variables X (llamadas también features)
y = df["MathScore"]
X = df.drop(["MathScore"], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## - LINEAR REGRESSION

1. Si usas modelos lineales (como regresión lineal o logística), una forma simple de reducir la multicolinealidad es eliminar una de las variables altamente correlacionadas.

In [127]:
# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
lm = linear_model.LinearRegression()
model= lm.fit(X_train, y_train)
print(f"R2: {round(lm.score(X,y), 3)}") # score es llamar a R2 // a partir de 0.7 bien
predictions = lm.predict(X_test)
mse = mean_squared_error(y_test, predictions) 
print(f"El MSE es: {mse}")

R2: 0.284
El MSE es: 165.01311955298223


## - RIDGE Y LASSO

2. Si prefieres mantener todas las variables en tu modelo, puedes usar un modelo de regresión con regularización como Ridge Regression (L2) o Lasso Regression (L1), que penaliza las variables con coeficientes grandes. Esto ayudará a manejar la multicolinealidad sin necesidad de eliminar columnas.

In [128]:
# Definir variables independientes (X) y dependientes (y)
y = df["MathScore"]
X = df.drop(["MathScore"], axis = 1)

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar los datos (importante para KNN, Lasso, Ridge)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenando el modelo Ridge
ridge = Ridge(alpha=0.1)
ridge.fit(X_train, y_train)
# Predicciones
y_pred_ridge = ridge.predict(X_test)
# Evaluación del modelo
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)
print("📘 Ridge:")
print(f"Mean Squared Error: {mse}")
print(f"R2 (Coeficiente de Determinación): {r2_ridge:.4f}\n")

# **3️⃣ Lasso (con penalización L1)**
lasso = Lasso(alpha=0.1)  # alpha controla la fuerza de la penalización. Cuanto más alpha, más se polariza la insignificancia de variables. 
# alpha=0 es lo mismo que hacer lineal. Más comunes: 0.0001, 0.001, 0.01, 0.1, 1
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)
print("📘 Lasso:")
print(f"MSE (Error Cuadrático Medio): {mse_lasso:.4f}")
print(f"R2 (Coeficiente de Determinación): {r2_lasso:.4f}\n")

📘 Ridge:
Mean Squared Error: 165.01311955298223
R2 (Coeficiente de Determinación): 0.2736

📘 Lasso:
MSE (Error Cuadrático Medio): 165.4102
R2 (Coeficiente de Determinación): 0.2719



## y que pasa si el modelo sigue una tendencia no lineal?

In [129]:
# Opción 1

from sklearn.preprocessing import PolynomialFeatures

degree=2
poly = PolynomialFeatures(degree=degree)
X_poly_train = poly.fit_transform(X_train)
X_poly_test = poly.transform(X_test)

In [130]:
# Opción 2

from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# Transformación polinómica de grado 3
poly_reg = make_pipeline(PolynomialFeatures(degree=2), LinearRegression())
poly_reg.fit(X_train, y_train)

y_pred = poly_reg.predict(X_test)

# Evaluar el rendimiento
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE (Error Cuadrático Medio): {mse:.4f}')
print(f'R2 (Coeficiente de Determinación): {r2:.4f}')

MSE (Error Cuadrático Medio): 165.2428
R2 (Coeficiente de Determinación): 0.2726


## - KNN

In [131]:
# ELIMINO LAS COLUMNAS NR SIBLINGS Y ISFIRSTCHILD
df.drop(columns= ["NrSiblings", "IsFirstChild_yes"], inplace=True, axis=1)
y = df["MathScore"]
X = df.drop(["MathScore"], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Crear y entrenar el modelo KNN Regressor
knn_model = KNeighborsRegressor(n_neighbors=5, weights='distance', metric="manhattan")  # n_neighbors=5 y pesos basados en la distancia
# Neighbors= Vecinos que se escogen para predecir. 
# Weights: puede ser distance o uniform, según si la importancia de los vecinos es igual o depende de cuánto de cerca esté
# Metric: Cómo se evalúa la distancia entre puntos. minkowski, manhattan, euclidean son los más comunes
knn_model.fit(X_train, y_train)

# Predicciones para el test
y_pred = knn_model.predict(X_test)

# funciona bien el modelo?
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE (Error Cuadrático Medio): {mse:.4f}')
print(f'R2 (Coeficiente de Determinación): {r2:.4f}')


MSE (Error Cuadrático Medio): 199.0724
R2 (Coeficiente de Determinación): 0.1237


In [132]:
metrics=["euclidean", "chebyshev", "manhattan"]
n=[3, 5, 7]
size=[0.1, 0.2, 0.3,  0.4]

In [133]:
acc=0
right_size=0.0
metric="aaa"
neighbors=17
for i in range(len(metrics)):
    for j in range(len(n)):
        for k in range(len(size)):
            X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=size[k], stratify=y, random_state=42)

            knn_model = KNeighborsRegressor(n_neighbors=n[j], metric=metrics[i]) 
            knn_model.fit(X_train, y_train)

            # Predicciones para el test
            y_pred = knn_model.predict(X_test)        

            # funciona bien el modelo?
            mse = mean_squared_error(y_test, y_pred)
            r2 = r2_score(y_test, y_pred)
            print(f"MSE of KNN regressor with {size[k]} Test_size, with {n[j]} number of neighbours and using the {metrics[i]} method:", mse)
            print(f"R2 of KNN regressor with {size[k]} Test_size, with {n[j]} number of neighbours and using the {metrics[i]} method:", r2)

            if r2>acc:
                acc=r2
                right_size=size[k]
                metric=metrics[i]
                neighbors=n[j]


print(acc, right_size, metric, neighbors)

MSE of KNN regressor with 0.1 Test_size, with 3 number of neighbours and using the euclidean method: 227.42173219772096
R2 of KNN regressor with 0.1 Test_size, with 3 number of neighbours and using the euclidean method: -0.007674864570117679
MSE of KNN regressor with 0.2 Test_size, with 3 number of neighbours and using the euclidean method: 221.46535192640565
R2 of KNN regressor with 0.2 Test_size, with 3 number of neighbours and using the euclidean method: 0.01590172737867246
MSE of KNN regressor with 0.3 Test_size, with 3 number of neighbours and using the euclidean method: 216.13131313131314
R2 of KNN regressor with 0.3 Test_size, with 3 number of neighbours and using the euclidean method: 0.04205479255680078
MSE of KNN regressor with 0.4 Test_size, with 3 number of neighbours and using the euclidean method: 213.37178701677607
R2 of KNN regressor with 0.4 Test_size, with 3 number of neighbours and using the euclidean method: 0.053865946972781464
MSE of KNN regressor with 0.1 Test_si

## - SVR

In [134]:
# Generar datos sintéticos

# Crear y entrenar un modelo SVR con kernel 'rbf' (para no linealidad). Puede ser 'linear', o 'poly'. 
# Para no lineal también se usa "sigmoid" para relaciones logísticas
svr = SVR(kernel='linear', C=10, epsilon=0.1) 
# C: Bajo vs alto: General (suave) o Ajuste preciso (ojo overfitting). 
# Poly: degree= Controla el grado de polinomio (2, 3, 4, 5)
# Gamma (todos menos linear): Controla la influencia de cada punto. gamma baja= puntos lejanos tienen más influencia (0.01). 
#Gamma alta(1, 10): Puntos cercanos tienen más influencia. Ojo con el sobreajuste. 'scale', 'auto', o 0.01, 0.1, 1
# Epsilon: Ignorar errores. Si es alto, ignora más

svr.fit(X_train, y_train)
# Predicciones
y_pred = svr.predict(X_test)
# Evaluar el rendimiento
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MSE (Error Cuadrático Medio): {mse:.4f}')
print(f'R2 (Coeficiente de Determinación): {r2:.4f}')

MSE (Error Cuadrático Medio): 160.7933
R2 (Coeficiente de Determinación): 0.2870


# OPCIÓN 2. TRATANDO CON LA MULTICOLINEALIDAD

## ME QUEDO CON EL WRITING SCORE    

In [135]:
df2 = pd.read_csv("StudentsMathWritingScoresAnalisis.csv")

In [136]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28561 entries, 0 to 28560
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype
---  ------               --------------  -----
 0   NrSiblings           28561 non-null  int64
 1   MathScore            28561 non-null  int64
 2   WritingScore         28561 non-null  int64
 3   ParentEduc           28561 non-null  int64
 4   PracticeSport        28561 non-null  int64
 5   WklyStudyHours       28561 non-null  int64
 6   Gender_male          28561 non-null  int64
 7   EthnicGroup_group B  28561 non-null  int64
 8   EthnicGroup_group C  28561 non-null  int64
 9   EthnicGroup_group D  28561 non-null  int64
 10  EthnicGroup_group E  28561 non-null  int64
 11  LunchType_standard   28561 non-null  int64
 12  TestPrep_none        28561 non-null  int64
 13  IsFirstChild_yes     28561 non-null  int64
dtypes: int64(14)
memory usage: 3.1 MB


OLS 

In [137]:
y = df2["MathScore"]
X = df2.drop(["MathScore"], axis = 1)
X = sm.add_constant(X) # añadir la constante (muy importante), no es lo mismo que corte en y=0 que en y=4
model = sm.OLS(y, X).fit()
# visualizar el modelo
print(model.summary())
# los coeficientes son las Betas, cómo el cambio de x afecta a la y
# p value > 0.05 no rechazamos la Hipotesis nula / menor de 0.05 sí rechazamos la hipotesis nula


                            OLS Regression Results                            
Dep. Variable:              MathScore   R-squared:                       0.863
Model:                            OLS   Adj. R-squared:                  0.863
Method:                 Least Squares   F-statistic:                 1.385e+04
Date:                Sat, 04 Jan 2025   Prob (F-statistic):               0.00
Time:                        14:44:36   Log-Likelihood:                -89509.
No. Observations:               28561   AIC:                         1.790e+05
Df Residuals:                   28547   BIC:                         1.792e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -9.3386    

PRUEBO A QUITAR NR OF SIBLINGS Y GRUPO B PORQUE ME DAN UN P VALUE MUY ALTO
ESTAS VARIABLES NO EXPLICAN EL MODELO

In [138]:
df2.drop(columns= ["NrSiblings", "EthnicGroup_group B", "IsFirstChild_yes"], inplace=True, axis=1)

In [139]:
y = df2["MathScore"]
X = df2.drop(["MathScore"], axis = 1)
X = sm.add_constant(X) # añadir la constante (muy importante), no es lo mismo que corte en y=0 que en y=4
model = sm.OLS(y, X).fit()
# visualizar el modelo
print(model.summary())
# los coeficientes son las Betas, cómo el cambio de x afecta a la y
# p value > 0.05 no rechazamos la Hipotesis nula / menor de 0.05 sí rechazamos la hipotesis nula


                            OLS Regression Results                            
Dep. Variable:              MathScore   R-squared:                       0.863
Model:                            OLS   Adj. R-squared:                  0.863
Method:                 Least Squares   F-statistic:                 1.800e+04
Date:                Sat, 04 Jan 2025   Prob (F-statistic):               0.00
Time:                        14:44:51   Log-Likelihood:                -89511.
No. Observations:               28561   AIC:                         1.790e+05
Df Residuals:                   28550   BIC:                         1.791e+05
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -9.2198    

comparitiva de grupos y de sexo

APLICO RIDGE Y LASSO

In [148]:
# Definir variables independientes (X) y dependientes (y)
y = df2["MathScore"]
X = df2.drop(["MathScore"], axis = 1)

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar los datos (importante para KNN, Lasso, Ridge)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenando el modelo Ridge
ridge = Ridge(alpha=1)
ridge.fit(X_train, y_train)
# Predicciones
y_pred_ridge = ridge.predict(X_test)
# Evaluación del modelo
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)
print("📘 Ridge:")
print(f"Mean Squared Error: {mse}")
print(f"R2 (Coeficiente de Determinación): {r2_ridge:.4f}\n")

# **3️⃣ Lasso (con penalización L1)**
lasso = Lasso(alpha=1)  # alpha controla la fuerza de la penalización. Cuanto más alpha, más se polariza la insignificancia de variables. 
# alpha=0 es lo mismo que hacer lineal. Más comunes: 0.0001, 0.001, 0.01, 0.1, 1
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)
print("📘 Lasso:")
print(f"MSE (Error Cuadrático Medio): {mse_lasso:.4f}")
print(f"R2 (Coeficiente de Determinación): {r2_lasso:.4f}\n")

📘 Ridge:
Mean Squared Error: 160.79325553429356
R2 (Coeficiente de Determinación): 0.8658

📘 Lasso:
MSE (Error Cuadrático Medio): 46.0527
R2 (Coeficiente de Determinación): 0.7973



## ME QUEDO CON LA MEDIA DE READINGSCORE Y WRITINGSCORE

In [149]:
df3 = pd.read_csv("StudentsMathandAVGScoresAnalisis.csv")

In [150]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28561 entries, 0 to 28560
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   NrSiblings           28561 non-null  int64  
 1   MathScore            28561 non-null  int64  
 2   AvgWritingReading    28561 non-null  float64
 3   ParentEduc           28561 non-null  int64  
 4   PracticeSport        28561 non-null  int64  
 5   WklyStudyHours       28561 non-null  int64  
 6   Gender_male          28561 non-null  int64  
 7   EthnicGroup_group B  28561 non-null  int64  
 8   EthnicGroup_group C  28561 non-null  int64  
 9   EthnicGroup_group D  28561 non-null  int64  
 10  EthnicGroup_group E  28561 non-null  int64  
 11  LunchType_standard   28561 non-null  int64  
 12  TestPrep_none        28561 non-null  int64  
 13  IsFirstChild_yes     28561 non-null  int64  
dtypes: float64(1), int64(13)
memory usage: 3.1 MB


OLS

In [151]:
y = df3["MathScore"]
X = df3.drop(["MathScore"], axis = 1)
X = sm.add_constant(X) # añadir la constante (muy importante), no es lo mismo que corte en y=0 que en y=4
model = sm.OLS(y, X).fit()
# visualizar el modelo
print(model.summary())
# los coeficientes son las Betas, cómo el cambio de x afecta a la y
# p value > 0.05 no rechazamos la Hipotesis nula / menor de 0.05 sí rechazamos la hipotesis nula

                            OLS Regression Results                            
Dep. Variable:              MathScore   R-squared:                       0.866
Model:                            OLS   Adj. R-squared:                  0.866
Method:                 Least Squares   F-statistic:                 1.415e+04
Date:                Sat, 04 Jan 2025   Prob (F-statistic):               0.00
Time:                        14:57:25   Log-Likelihood:                -89245.
No. Observations:               28561   AIC:                         1.785e+05
Df Residuals:                   28547   BIC:                         1.786e+05
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                 -10.0848    

In [152]:
df3.drop(columns= ["NrSiblings", "EthnicGroup_group B"], inplace=True, axis=1)

In [153]:
y = df3["MathScore"]
X = df3.drop(["MathScore"], axis = 1)
X = sm.add_constant(X) # añadir la constante (muy importante), no es lo mismo que corte en y=0 que en y=4
model = sm.OLS(y, X).fit()
# visualizar el modelo
print(model.summary())
# los coeficientes son las Betas, cómo el cambio de x afecta a la y
# p value > 0.05 no rechazamos la Hipotesis nula / menor de 0.05 sí rechazamos la hipotesis nula


                            OLS Regression Results                            
Dep. Variable:              MathScore   R-squared:                       0.866
Model:                            OLS   Adj. R-squared:                  0.866
Method:                 Least Squares   F-statistic:                 1.672e+04
Date:                Sat, 04 Jan 2025   Prob (F-statistic):               0.00
Time:                        14:57:32   Log-Likelihood:                -89246.
No. Observations:               28561   AIC:                         1.785e+05
Df Residuals:                   28549   BIC:                         1.786e+05
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                 -10.0182    

APLICO RIDGE Y LASSO

In [156]:
# Definir variables independientes (X) y dependientes (y)
y = df3["MathScore"]
X = df3.drop(["MathScore"], axis = 1)

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Escalar los datos (importante para KNN, Lasso, Ridge)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenando el modelo Ridge
ridge = Ridge(alpha=1)
ridge.fit(X_train, y_train)
# Predicciones
y_pred_ridge = ridge.predict(X_test)
# Evaluación del modelo
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)
print("📘 Ridge:")
print(f"Mean Squared Error: {mse}")
print(f"R2 (Coeficiente de Determinación): {r2_ridge:.4f}\n")

# **3️⃣ Lasso (con penalización L1)**
lasso = Lasso(alpha=0.1)  # alpha controla la fuerza de la penalización. Cuanto más alpha, más se polariza la insignificancia de variables. 
# alpha=0 es lo mismo que hacer lineal. Más comunes: 0.0001, 0.001, 0.01, 0.1, 1
lasso.fit(X_train, y_train)
y_pred_lasso = lasso.predict(X_test)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)
print("📘 Lasso:")
print(f"MSE (Error Cuadrático Medio): {mse_lasso:.4f}")
print(f"R2 (Coeficiente de Determinación): {r2_lasso:.4f}\n")

📘 Ridge:
Mean Squared Error: 160.79325553429356
R2 (Coeficiente de Determinación): 0.8685

📘 Lasso:
MSE (Error Cuadrático Medio): 30.1569
R2 (Coeficiente de Determinación): 0.8672

